# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

output_data_file_part2 = '/Users/royjiang/Desktop/UCI-VIRT-DATA-PT-10-2022-U-LOLC/module6startercode/Instructions/output_data/city_weather_part1_data.csv'
output_data_part2 = pd.read_csv(output_data_file_part2)
output_data_part2.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country_Date
0,Kapaa,22.0752,-159.3190,80.24,74,40,17.27,1668989700
1,Pacific Grove,36.6177,-121.9166,62.91,56,0,3.44,1668989700
2,Tasiilaq,65.6145,-37.6368,39.36,81,100,16.64,1668989701
3,Rørvik,64.8621,11.2373,19.51,86,2,0.00,1668989702
4,Ushuaia,-54.8000,-68.3000,46.06,71,40,8.05,1668989701


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
location = output_data_part2[['Lat', 'Lng']].astype(float)
location

,Lat,Lng
0,22.0752,-159.3190
1,36.6177,-121.9166
2,65.6145,-37.6368
3,64.8621,11.2373
4,-54.8000,-68.3000
...,...,...
563,32.4488,-81.7832
564,-22.4500,18.9667
565,31.3525,27.2453
566,58.1394,52.6584


In [4]:
gmaps.configure(api_key=g_key)

In [5]:
output_data_part2 = output_data_part2.dropna()
humidity = output_data_part2["Humidity"].astype(float)

In [7]:
figure_layout = {
'width': '500px',
'height': '500px',
'border': '1px solid black',
'padding': '1px'
}
fig = gmaps.figure(layout=figure_layout)
heatmap_layer = gmaps.heatmap_layer(location, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', padding='1px', width='500px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
# A max temperature lower than 70 degrees but higher than 50.
output_data_part2 = output_data_part2.loc[(output_data_part2["Max_Temp"] < 70) & (output_data_part2["Max_Temp"] > 50)]

# Wind speed less than 10 mph but greater than 5.
output_data_part2 = output_data_part2.loc[(output_data_part2["Wind_Speed"] < 10) & (output_data_part2["Wind_Speed"] >5)]
# humidity greater than 80.
output_data_part2 = output_data_part2.loc[output_data_part2["Humidity"] > 80]

# Drop any rows with null values
output_data_part2 = output_data_part2.dropna()

output_data_part2

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country_Date
10,Dolores,-36.3132,-57.6792,61.36,97,100,8.28,1668989705
34,Port Alfred,-33.5906,26.8910,58.17,93,17,7.31,1668989715
38,East London,-33.0153,27.9116,61.50,81,12,8.95,1668989716
39,Castro,-24.7911,-50.0119,62.17,93,74,7.90,1668989717
65,Katsuura,35.1333,140.3000,59.74,94,100,7.94,1668989727
112,Hermanus,-34.4187,19.2345,54.81,82,2,7.05,1668989742
164,Hualmay,-11.0964,-77.6139,65.08,83,86,7.25,1668989761
210,Babayurt,43.6002,46.7789,52.39,82,87,8.97,1668989779
238,Gunjur,13.2019,-16.7339,69.87,94,0,6.91,1668989789
314,Coquimbo,-29.9533,-71.3436,58.73,88,0,8.05,1668989816


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = output_data_part2

In [16]:
import json

In [22]:
# params dictionary to update each iteration
params = {
    "radius": 10000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel_address = requests.get(base_url, params=params)
    hotel_address = hotel_address.json()
    result = hotel_address['results']
#    print(json.dumps(hotel_address, indent=4, sort_keys=True))
    try:
        print(f"Closest hotel is {result[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = result[0]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Closest hotel is Howard Johnson By Wyndham Termal Dolores.
Closest hotel is MyPond Hotel.
Closest hotel is Blue Lagoon Hotel.
Closest hotel is Hotel Borgen.
Closest hotel is Katsuura Hotel Mikazuki.
Closest hotel is Harbour House Hotel.
Closest hotel is Hotel Terrazas Del Mar - Huacho.
Closest hotel is Комната для намаза.
Closest hotel is The Gunjur Project Lodge.
Closest hotel is Hotel Club La Serena.
Closest hotel is Hotel Master Plaza.
Closest hotel is Shell Jiangsu Ancheng City Sheyang County Sun City.
Closest hotel is Dormy Inn Izumo.
Closest hotel is Hotel New Palace.
Closest hotel is LABEL VILLAGE ARIVONIMAMO.
Closest hotel is Holiday Inn Buenos Aires Ezeiza Airport, an IHG Hotel.


In [23]:
 hotel_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country_Date,Hotel Name
10,Dolores,-36.3132,-57.6792,61.36,97,100,8.28,1668989705,Howard Johnson By Wyndham Termal Dolores
34,Port Alfred,-33.5906,26.8910,58.17,93,17,7.31,1668989715,MyPond Hotel
38,East London,-33.0153,27.9116,61.50,81,12,8.95,1668989716,Blue Lagoon Hotel
39,Castro,-24.7911,-50.0119,62.17,93,74,7.90,1668989717,Hotel Borgen
65,Katsuura,35.1333,140.3000,59.74,94,100,7.94,1668989727,Katsuura Hotel Mikazuki
112,Hermanus,-34.4187,19.2345,54.81,82,2,7.05,1668989742,Harbour House Hotel
164,Hualmay,-11.0964,-77.6139,65.08,83,86,7.25,1668989761,Hotel Terrazas Del Mar - Huacho
210,Babayurt,43.6002,46.7789,52.39,82,87,8.97,1668989779,Комната для намаза
238,Gunjur,13.2019,-16.7339,69.87,94,0,6.91,1668989789,The Gunjur Project Lodge
314,Coquimbo,-29.9533,-71.3436,58.73,88,0,8.05,1668989816,Hotel Club La Serena


In [26]:
locations = hotel_df[["Lat", "Lng"]]

In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """


Hotel Name
      {Hotel Name}
City
  {City}
Country_Date
  {Country_Date} 

"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [33]:
markers = gmaps.marker_layer(locations)

In [39]:
# Add marker layer ontop of heat map
# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='red',
    stroke_color='black', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='500px', padding='1px', width='500px'))